<h1><center>  lab 9 : Text Classification </center>
    
<img src="https://miro.medium.com/max/1200/0*kHsYtU2d9IgyFwet.jpg" width="500">


```Created by Jinnie Shin (jinnie.shin@coe.ufl.edu)```\
```Date: May 17 2022```

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQmNf86oJnfhpkPA9LnrFnAbfwF2VywPYpB_w&usqp=CAU" align="left" width="70" height="70" align="left"> 

 ### Required Packages or Dependencies

In [ ]:
#!pip install { } ! in case you run into the `package not avaialble` error
import pandas as pd 
import numpy as np 


## Dataset
#### Essay Set 10
---
`Prompt`: \
Brandi and Jerry were designing a doghouse. Brandi and Jerry did the following controlled experiment to find out how the color of an object affects its temperature. Use the results from the experiment to describe the best paint color for the doghouse. 
In your description, be sure to:\
•	Choose a paint color. \
•	Describe how that color might affect the inside of the doghouse. \
•	Use results from the experiment to support your description.


> `1. Question`: What is the effect of different lid colors on the air temperature inside a glass jar exposed to a lamp? \
> `2. Hypothesis`: The darker the lid color, the greater the increase in air temperature in the glass jar, as darker colors absorb more energy. \
> `3. Materials`:
glass jar, lamp, four colored lids: {black, dark gray, light gray, and white}, thermometer, meterstick, stopwatch\
> `4. Procedures`:
>> 1.	Put the black lid with the attached thermometer on the glass jar.
>> 2.	Make sure the starting temperature inside the jar is 24° C.
>> 3.	Place lamp 5 centimeters away from the lid and turn on the lamp.
>> 4.	After 10 minutes measure the air temperature inside the glass jar and record as Trial 1.
>> 5.	Turn off lamp and wait until the air in the jar returns to the starting temperature.
>> 6.	Repeat steps 2 through 5 for Trials 2 and 3.
>> 7.	Repeat steps 1 through 6 for the dark gray, light gray, and white lids.
>> 8.	Calculate and record the average air temperature for each lid color. 

> `5. Results`: Lid Colour vs. Air Temperature (Insides Glass Jar After 10 mins (°C)

| Lid Colour |   Trial 1|  Trial 1|   Trial 3| Average|
|------------|----------|----------|----------|----------|
| Black      | `54°C`   |`52°C`   |`54°C`   |`53°C`   |
| Dark gray  | `48°C`   | `48°C`   |`48°C`   | `48°C`   |
| Light gray | `44°C`   |`44°C`   |`46°C`   |`45°C`   |
| White      | `42°C`   | `43°C`   |`41°C`   | `42°C`   |

---
#### Essay Set 10 Data Description

| Type of response            | Source dependent response |
|-----------------------------|---------------------------|
| Grade level                 | `8`                       |
| Subject                     | `Science`                   |
| Total sample size           | `1,640`                     |
| Average length of responses | `60 words`                  |
| Score range                 | `0-2`                       |




In [ ]:
df = pd.read_csv('./data/train_rel_2.tsv', sep='\t')
df = df[df['EssaySet'] == 10]
df[:5]

,Id,EssaySet,Score1,Score2,EssayText
15403,25949,10,2,2,black :: black might affect the dog house beca...
15404,25950,10,1,1,white :: White because you wouldn't want the d...
15405,25951,10,2,2,white :: the color white will reflect the suns...
15406,25952,10,1,1,white :: this color will keep the dog house co...
15407,25953,10,2,2,white :: I would use a light color like White ...


<img src="https://i.pinimg.com/736x/2e/aa/7d/2eaa7d5021ca7c3c98bc93b98b9646fe.jpg" align="left" width="70" height="70" align="left">

 ## Task 1: Training data
>  Q1. Let's create a new column, called `color`, in the dataframe `df` that only saves the student's reponses regarding the colour choice (e.g., black, white, ... )  \
Hint: You can use the functions `.split()` and `.apply()`

> Q2: Remove the colour choice response from the `EssayText` column and save it into a new column `response`  \
(e.g., "black :: black might affect the dog ~ " -> "black might affect the dog~")

In [ ]:
################################### YOUR CODE HERE for Q1 #############################
df['color'] = df.EssayText.apply(lambda x: x.split('::')[0])

#######################################################################################
df['color'].value_counts()

white          733
black          392
light gray     288
dark gray      225
                 2
Name: color, dtype: int64

In [ ]:
################################### YOUR CODE HERE for Q2 #############################
df['response'] = df.EssayText.apply(lambda x: x.split('::')[1] if '::' in x else x)

#######################################################################################
df.response

15403     black might affect the dog house because then...
15404     White because you wouldn't want the dog to ge...
15405     the color white will reflect the suns rays of...
15406     this color will keep the dog house cooler ins...
15407     I would use a light color like White in this ...
                               ...                        
17038     white becuase if you live in a hot place the ...
17039     This color will affect the doghouse for the b...
17040     i think light gray would work the best becaus...
17041      if they painted the doghouse dark gray, then...
17042     the black would affect the doghouse because t...
Name: response, Length: 1640, dtype: object


## **REVIEW**: Text Vectorizer (from Week 4)

> We will use the coh-metrix indices introduced in Week 6, `features.xlsx`

In [ ]:
class text_vectorizer():
    
    def count_vectorizer(self, data): #Input is the dataframe we created above with three columns 
        self.data = data
        
        from collections import Counter # import the Counter function 
        import pandas as pd 
        import nltk 
        
        self.data['new_response'] = self.data['response'].apply(nltk.word_tokenize)
        self.data['new_response'] = self.data['new_response'].apply(Counter).apply(dict) #we apply the counter function to the "cleaned" columns
                                                                              # we used the apply function in week 2

        self.bow = pd.DataFrame(self.data['new_response'].tolist()).T.fillna(0) # we restructure the output as a dataframe and name it "bow"
                                                                      # also fill in the missing values (nan -> 0 )
        self.unique_vocab = list(self.bow.index)                                # FYI, bow standards for (bag-of-words)

        return self.bow, self.unique_vocab

    def tfidf_vectorizer(self, data):
        self.data = data
        
        import pandas as pd 
        for response in self.bow.columns:
            self.bow[response] = self.bow[response].apply(lambda x: np.log10(x+1))

        df = pd.DataFrame( ( ( bow > 0 ) * 1 ).sum(axis=1), columns=['df'] )  #this is the document-freqeuncy 
        N = self.bow.shape[1] # this is the collection frequency (total number of doucments)
        df['IDF'] = df['df'].apply(lambda x: np.log10(N/x))

        self.tfidf = bow.multiply(df.IDF, axis='index')

        return self.tfidf


In [ ]:
vect = text_vectorizer()
bow, vocab = vect.count_vectorizer(df)
bow[:5]
X= bow.T
y = df.Score1
print(y.value_counts())

1    770
2    580
0    290
Name: Score1, dtype: int64


## 1. Naive Bayes Classifier

In [ ]:
class NB():
    
    def __init__(self):
        # count is a dictionary which stores several dictionaries corresponding to each news category
        # each value in the subdictionary represents the freq of the key corresponding to that news category 
        self.doc_count ={}
        self.word_count = {}
        self.classes = None
        self.vocab = []
    
    def fit(self,X_train,Y_train):
        
        self.vocab = list(X_train.columns)
        self.doc_count = dict(Y_train.value_counts())
        self.classes = list(set(Y_train))
        
        data = X_train
        data['output_y'] = Y_train.tolist()
        
        for class_ in self.classes: 
            self.word_count[class_] = dict(data[data['score'] ==class_].drop(columns =['output_y']).sum())
        
        self.doc_count['total'] = sum(self.doc_count.values())
        self.word_count['total']= dict(X_train.sum())
        
    def probability(self,test_vec,class_):
        
        log_prob = np.log(self.doc_count[class_]) - np.log(self.doc_count['total'])
        total_words = len(test_vec.index)
        for i in list(test_vec.index):
            try: 
                current_word_prob = test_vec[i]*(np.log(self.word_count[class_][i]+1)-np.log(self.word_count['total'][i]+total_words))
                log_prob += current_word_prob
            except:
                log_prob = log_prob 
        return log_prob
    
    
    def predictSinglePoint(self,test_point):
        
        
        best_class = None
        best_prob = None
        first_run = True
        
        for class_ in self.classes:
            log_probability_current_class = self.probability(test_point,class_)
            if (first_run) or (log_probability_current_class > best_prob) :
                best_class = class_
                best_prob = log_probability_current_class
                first_run = False
                
        return best_class
    
    def predict(self,X_test):
        # This can take some time to complete
        Y_pred = [] 
        for i in range(len(X_test)):
        # print(i) # Uncomment to see progress
            Y_pred.append( self.predictSinglePoint(X_test.iloc[i]) )
        
        return Y_pred
    
    def score(self,Y_pred,Y_true):
        # returns the mean accuracy
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y_true[i]:
                count+=1
        return count/len(Y_pred)

In [ ]:
nbc = NB()
nbc.fit(X, y)

In [ ]:
nbc.doc_count

{1: 770, 2: 580, 0: 290, 'total': 1640}

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = MultinomialNB()
clf.fit(X,y)
Y_test_pred = clf.predict(X)
sklearn_score_train = clf.score(X,y)
print("Sklearn's score on training data :",sklearn_score_train)
sklearn_score_test = clf.score(X,y)
print("Sklearn's score on testing data :",sklearn_score_test)
print("Classification report for testing data :-")
print(classification_report(y, Y_test_pred))


Sklearn's score on training data : 0.8073170731707318
Sklearn's score on testing data : 0.8073170731707318
Classification report for testing data :-
              precision    recall  f1-score   support

           0       1.00      0.45      0.62       290
           1       0.77      0.91      0.83       770
           2       0.83      0.86      0.84       580

    accuracy                           0.81      1640
   macro avg       0.86      0.74      0.76      1640
weighted avg       0.83      0.81      0.80      1640

